In [109]:
%pip install transformers datasets evaluate accelerate peft tqdm ipywidgets torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
hugging_face_token = ""

In [ ]:
%pip install huggingface_hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
from huggingface_hub import notebook_login

notebook_login()


In [2]:
import torch
from transformers import RobertaModel, RobertaTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer
from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification
from datasets import load_dataset, Dataset
import pandas as pd
from torch.utils.data import DataLoader
import evaluate


In [3]:
neologism_data = pd.read_csv('all_sampled_twitter.csv')
neologism_data['label'] = neologism_data['label'].astype("int64")

dataset = Dataset.from_pandas(neologism_data)

train_test_split = dataset.train_test_split(test_size=0.3, seed=42)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]
eval_dataset = train_test_split["test"]

In [4]:
neologism_data

,text,label
0,"""@mattduss @chrislhayes if ISIS does gain a fo...",0
1,"""Sad news! 150 people shot, drowned in Yobe ri...",0
2,@StephenNolan they all served and died under t...,0
3,Rousey vs. Holm may be the biggest waste of a ...,0
4,@user Amy Schumer is the stereotypical 1st wor...,0
...,...,...
8739,The city experienced a complete blackout after...,0
8740,"After the power outage, the city experienced a...",0
8741,The unexpected blackout plunged the city into ...,0
8742,The company's sudden mobilization of resources...,0


In [5]:
base_model = f'cardiffnlp/twitter-roberta-base-sentiment-latest'

device = "cuda"

num_labels = 3
class_names = ['negative', 'neutral', 'positive']

id2label = {i: label for i, label in enumerate(class_names)}

In [6]:
tokenizer_full_finetune = RobertaTokenizer.from_pretrained(base_model)
model_to_train = AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label, num_labels=num_labels).to(device)

def tokenize_function(examples):
    return tokenizer_full_finetune(examples["text"], padding="max_length", truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True).with_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset = test_dataset.map(tokenize_function, batched=True).with_format("torch", columns=["input_ids", "attention_mask", "label"])
eval_dataset = eval_dataset.map(tokenize_function, batched=True).with_format("torch", columns=["input_ids", "attention_mask", "label"])


data_collator = DataCollatorWithPadding(tokenizer=tokenizer_full_finetune, return_tensors="pt")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Map:   0%|          | 0/6120 [00:00<?, ? examples/s]

Map:   0%|          | 0/2624 [00:00<?, ? examples/s]

Map:   0%|          | 0/2624 [00:00<?, ? examples/s]

In [7]:
training_args = TrainingArguments(
    eval_strategy='steps',
    learning_rate=5e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    fp16=torch.cuda.is_available(),
    output_dir="AkhilaGP/roberta-senti-neologism-full-finetune",
    push_to_hub=True
)

def get_trainer(model):
      return  Trainer(
          model=model,
          args=training_args,
          train_dataset=train_dataset,
          eval_dataset=eval_dataset,
          data_collator=data_collator,
      )

In [8]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def evaluate_model(inference_model, dataset):

    predictions_full = []
    references_full = []
    eval_dataloader = DataLoader(dataset.rename_column("label", "labels"), batch_size=8, collate_fn=data_collator)
    inference_model.to("cuda")
    inference_model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch.to("cuda")
        with torch.no_grad():
            outputs = inference_model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        predictions, references = predictions, batch["labels"]
        predictions_full.append(predictions)
        references_full.append(references)
        accuracy_metric.add_batch(predictions=predictions,references=references,)
        f1_metric.add_batch(predictions=predictions,references=references,)
        precision_metric.add_batch(predictions=predictions,references=references,)
        recall_metric.add_batch(predictions=predictions,references=references,)

    accuracy = accuracy_metric.compute()
    f1 = f1_metric.compute(average="weighted")
    precision = precision_metric.compute(average="weighted")
    recall = recall_metric.compute(average="weighted")

    print(f"Accuracy: {accuracy}")
    print(f"F1 Score (macro): {f1}")
    print(f"Precision (macro): {precision}")
    print(f"Recall (macro): {recall}")
    return predictions_full, references_full

In [10]:
full_finetuning_trainer = get_trainer(model_to_train)
full_finetuning_trainer.train()
# full_finetuning_trainer.push_to_hub()
# tokenizer_full_finetune.push_to_hub("roberta-senti-neologism-full-finetune")

README.md:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AkhilaGP/roberta-senti-neologism-full-finetune/commit/1ae917bf9c457331492c5fa88be54a1e0f99880a', commit_message='Upload tokenizer', commit_description='', oid='1ae917bf9c457331492c5fa88be54a1e0f99880a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AkhilaGP/roberta-senti-neologism-full-finetune', endpoint='https://huggingface.co', repo_type='model', repo_id='AkhilaGP/roberta-senti-neologism-full-finetune'), pr_revision=None, pr_num=None)

In [9]:
from tqdm import tqdm
evaluate_model(AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label, num_labels=num_labels).to(device), test_dataset)
pred, ref = evaluate_model(AutoModelForSequenceClassification.from_pretrained("AkhilaGP/roberta-senti-neologism-full-finetune", id2label=id2label, num_labels=num_labels).to(device), test_dataset)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 328/328 [00:25<00:00, 12.90it/s]


Accuracy: {'accuracy': 0.7545731707317073}
F1 Score (macro): {'f1': 0.7621121659307027}
Precision (macro): {'precision': 0.7784456828044978}
Recall (macro): {'recall': 0.7545731707317073}


100%|██████████| 328/328 [00:24<00:00, 13.17it/s]


Accuracy: {'accuracy': 0.7736280487804879}
F1 Score (macro): {'f1': 0.7729581944188018}
Precision (macro): {'precision': 0.7723608657814851}
Recall (macro): {'recall': 0.7736280487804879}


In [10]:
def classify(text, model, tokenizer):
  inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt").to(device)
  output = model(**inputs)
  prediction = output.logits.argmax(dim=-1).item()
  print(f'Sentiment: {id2label[prediction]} for {text}')

In [11]:
def classify_og(text, model, tokenizer):
  inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt").to(device)
  output = model(**inputs)
  prediction = output.logits.argmax(dim=-1).item()
  print(f'Sentiment: {id2label[prediction]} for {text}')

In [12]:
samples = [    "Listening to the retro playlist filled with 80's synth-pop hits, he was filled by a wave of falstalagia",
        "That fit is straight fire, no cap, you're looking mad schmick",
        "The candidate's speech was pure clickbait, all sizzle and no steak",
        "The workshop was absolute chaos, with ideas flying in every direction, but somehow, in the middle of that whirlwind, we created something ridiculously inspiring.",
        "Absolutely love how our talent pool is just bursting with sparkle-genius nepo babies, each one more effortlessly deserving and glow-ready for success than anyone who actually worked for it.",
        "That team-building session was an absolute trainwreck of synergy—everyone was crushed by inspiration and overloaded with motivation to the point of absurdity",
        "Taylor's new album slaps hard"]
        

non_finetuned = AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label, num_labels=num_labels).to(device)
full_finetuned = AutoModelForSequenceClassification.from_pretrained("AkhilaGP/roberta-senti-neologism-full-finetune", id2label=id2label, num_labels=num_labels).to(device)

for i in samples:
    classify(i, non_finetuned , tokenizer_full_finetune )

print("Full")
for i in samples:
    classify(i, full_finetuned, tokenizer_full_finetune)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentiment: neutral for Listening to the retro playlist filled with 80's synth-pop hits, he was filled by a wave of falstalagia
Sentiment: positive for That fit is straight fire, no cap, you're looking mad schmick
Sentiment: negative for The candidate's speech was pure clickbait, all sizzle and no steak
Sentiment: positive for The workshop was absolute chaos, with ideas flying in every direction, but somehow, in the middle of that whirlwind, we created something ridiculously inspiring.
Sentiment: positive for Absolutely love how our talent pool is just bursting with sparkle-genius nepo babies, each one more effortlessly deserving and glow-ready for success than anyone who actually worked for it.
Sentiment: negative for That team-building session was an absolute trainwreck of synergy—everyone was crushed by inspiration and overloaded with motivation to the point of absurdity
Sentiment: positive for Taylor's new album slaps hard
Full
Sentiment: positive for Listening to the retro playlist

LORA on roberta-base

In [13]:
training_args_LORA = TrainingArguments(
    eval_strategy='steps',
    learning_rate=5e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    fp16=torch.cuda.is_available(),
    output_dir="AkhilaGP/roberta-senti-neologism-LORA",
    push_to_hub=True

)

def get_trainer_LORA(model):
      return  Trainer(
          model=model,
          args=training_args_LORA,
          train_dataset=train_dataset,
          eval_dataset=eval_dataset,
          data_collator=data_collator,
      )

In [15]:
peft_config = LoraConfig(task_type="SEQ_CLS", 
                         inference_mode=False, 
                         r=8, 
                         lora_alpha=16, 
                         lora_dropout=0.1)

peft_model = get_peft_model(AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label).to(device), 
                            peft_config)

peft_model.print_trainable_parameters()

peft_lora_finetuning_trainer = get_trainer_LORA(peft_model)

peft_lora_finetuning_trainer.train()
peft_lora_finetuning_trainer.push_to_hub()
tokenizer_full_finetune.push_to_hub("roberta-senti-neologism-LORA")


README.md:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AkhilaGP/roberta-senti-neologism-LORA/commit/a2cc36150be77fe25e566cf1516d58ae6ed00171', commit_message='Upload tokenizer', commit_description='', oid='a2cc36150be77fe25e566cf1516d58ae6ed00171', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AkhilaGP/roberta-senti-neologism-LORA', endpoint='https://huggingface.co', repo_type='model', repo_id='AkhilaGP/roberta-senti-neologism-LORA'), pr_revision=None, pr_num=None)

In [16]:
roberta_lora = AutoPeftModelForSequenceClassification.from_pretrained("AkhilaGP/roberta-senti-neologism-LORA", id2label=id2label)
tokenizer_lora = AutoTokenizer.from_pretrained("AkhilaGP/roberta-senti-neologism-LORA")
pred, ref = evaluate_model(roberta_lora, test_dataset)

adapter_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

100%|██████████| 328/328 [00:26<00:00, 12.56it/s]

Accuracy: {'accuracy': 0.7930640243902439}
F1 Score (macro): {'f1': 0.785750786125696}
Precision (macro): {'precision': 0.7852043080500367}
Recall (macro): {'recall': 0.7930640243902439}


In [17]:
for i in samples:
    classify(i, roberta_lora , tokenizer_lora)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentiment: positive for Listening to the retro playlist filled with 80's synth-pop hits, he was filled by a wave of falstalagia
Sentiment: positive for That fit is straight fire, no cap, you're looking mad schmick
Sentiment: negative for The candidate's speech was pure clickbait, all sizzle and no steak
Sentiment: positive for The workshop was absolute chaos, with ideas flying in every direction, but somehow, in the middle of that whirlwind, we created something ridiculously inspiring.
Sentiment: positive for Absolutely love how our talent pool is just bursting with sparkle-genius nepo babies, each one more effortlessly deserving and glow-ready for success than anyone who actually worked for it.
Sentiment: negative for That team-building session was an absolute trainwreck of synergy—everyone was crushed by inspiration and overloaded with motivation to the point of absurdity
Sentiment: positive for Taylor's new album slaps hard
